In [1]:
print('hi')

hi


## Import Section

In [2]:
import numpy as np
import torch
import torchvision
from time import time
import random

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.autograd import Variable
import os
import glob
import cv2
from tqdm import tqdm

from matplotlib.pyplot import imsave, imread
import matplotlib.pyplot as plt
import sys
import matplotlib.gridspec as gridspec

import copy

In [3]:
from floodfill import *
from dataloader import *
from model import *
from oracle import *

## Testing Jupyter Notebook Section

In [ ]:
#Testing live input from user
testing_input = input("what is your name: ")
print(f"Your name is: {testing_input}.")

In [ ]:
#Tests loading an image from a directory
#im_dir = "/usr/xtmp/mammo/image_datasets/data_split_july2021/square_ROI_by_shape_segmentations/train/Round/"
#im_dir = "/usr/xtmp/vs196/mammoproj/Code/ActiveLearning/UNetSegmentations/Iter0_ff/Round/"
im_dir = "/usr/xtmp/vs196/mammoproj/Code/ActiveLearning/UNetSegmentations/Iter0_ff/Irregular/"
#ims = ["DP_AABO_1432_1.npy"]
#ims = ["DP_AHTL_130540_1.npy"]

for im in ims:
    path = im_dir + im;
    arr = np.load(path)
    print(arr.shape)
    
plt.imshow(arr[0],cmap='gray')
plt.show()

## Active Learning: Setup

In [ ]:
#Gets the dataloader. Passes in directory of ALL images
#im_dir = "/usr/xtmp/mammo/image_datasets/data_split_july2021/square_ROI_by_shape_segmentations/train/"
im_dir = "/usr/xtmp/vs196/mammoproj/Code/ActiveLearning/UNetSegmentations/Iter0_ff/"
dataloader = get_DataLoader(im_dir,32,2)

#Tests the dataloader
for i in tqdm(dataloader):
    #print(i[1])
    print(i[1].shape)
    break

In [ ]:
#Initializes and trains the model. Plots a loss function of the initial training
model,loss_tracker,criterion,optimizer = initialize_and_train_model(dataloader, epochs=5) #default batch_size and epochs
plt.plot(loss_tracker) #plot graph

In [ ]:
#Initializing info arrays for active learning
oracle_results = {}
all_patient_scores = []

#right now patient_scores is initial scores for each patient (without active learning training)
patient_scores = get_patient_scores(model,dataloader) 
all_patient_scores.append(patient_scores)

## Active Learning: Querying

In [ ]:
#Query oracle and update model via active learning
oracle_results = query_oracle(oracle_results,patient_scores,im_dir)

model = model_update(model,dataloader,oracle_results,criterion,optimizer,num_epochs=5)

patient_scores = get_patient_scores(model,dataloader)
all_patient_scores.append(patient_scores)

## Metrics and plotting

In [ ]:
#00100001000
#11110100000
#00100001100
#01110010000
#Prints out all patient scores.
for i in all_patient_scores:
    print(calculate_dispersion_metric(i,oracle_results))

In [ ]:
#Plot the disperson metric
j = []
for i in all_patient_scores:
    j.append(calculate_dispersion_metric(i,oracle_results))
    
plt.plot(j)

## Saving Oracle Results

In [ ]:
#make filepath
# save_dir = "/usr/xtmp/vs196/mammoproj/Code/ActiveLearning/AllOracleRuns/11_15_run/Iter1/"
# saved_oracle_filepaths = save_oracle_results(oracle_results,im_dir,save_dir)
#TODO: Move border/weight creation to this method and out of convert_directory_to_floodfill
#TODO: Remove all 0's from oracle_results. 
#NOTE: Don't FULLY reset oracle_results EVER, only do above

## Retrain UNet and Save New Segmentations

In [ ]:
#Define the initial UNet model (iter0)
model_path = "/usr/xtmp/vs196/mammoproj/SavedModels/HyperparameterUNet_nobuffer/unet_5_0.5/Model/unetmodel_FINAL.pth"
unet_model = torch.load(model_path)

total_loss_tracker = []
total_metric_tracker = []

In [ ]:
#Retrain UNet with training data taken from OracleImages/Iter
new_unet_dataloader = unet_dataloader(saved_oracle_filepaths,32,2)

unet_model,loss_tracker,metric_tracker = update_unet_model(unet_model,dataloader,num_epochs=10)
total_loss_tracker.extend(loss_tracker)
total_metric_tracker.extend(metric_tracker)

#evaluate new segmentations for ALL images 
#Save SAVED_ORACLE_FILEPATH NEW segmentations to SEPARATE FOLDER for VIEWING ONLY
#Save this iteration saved_oracle_filepaths along with new segmentations for all other patients


In [4]:
#Convert initial UNet Segmentations to flood-filled and resave
im_dir = "/usr/xtmp/mammo/image_datasets/data_split_july2021/square_ROI_by_shape_segmentations/train/"
new_im_dir = convert_directory_to_floodfill(im_dir,iter0=True)

100%|██████████| 824/824 [05:48<00:00,  2.36it/s]


In [ ]:
print(new_im_dir)